# PROJECT SOLUTION

# TASK #1: IMPORT KEY LIBRARIES/DATASETS, PERFORM EDA AND PREPARE THE DATA FOR TRAINING

In [1]:
# Import Numpy and check the version
import numpy as np
print(np.__version__)

1.21.6


In [2]:
# Import Numpy and check the version
import pandas as pd
print(pd.__version__)

1.3.5


In [3]:
# Updgrade Numpy version
!pip3 install numpy --upgrade

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
# Updgrade Pandas version
!pip3 install pandas --upgrade

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px # Interactive Data Visualization

In [6]:
# Read the CSV file 
stock_df = pd.read_csv("S&P500_Stock_Data.csv")

In [7]:
# Load the top 6 instances
stock_df.head(6)

,Interest Rates,Employment,S&P 500 Price
0,1.943859,55.413571,2206.680582
1,2.258229,59.546305,2486.474488
2,2.215863,57.414687,2405.868337
3,1.977960,49.908353,2140.434475
4,2.437723,52.035492,2411.275663
5,2.143637,56.060598,2187.344909


In [8]:
# Load the bottom 6 instances 
stock_df.tail(6)

,Interest Rates,Employment,S&P 500 Price
994,1.868835,60.543377,2065.424594
995,2.528420,52.390828,2578.699184
996,2.643571,54.626204,2573.865933
997,2.009644,57.004357,2004.911218
998,2.201656,62.447790,2344.098981
999,2.180638,46.297034,2156.133786


In [9]:
# Display the feature columns
stock_df.columns

Index(['Interest Rates', 'Employment', 'S&P 500 Price'], dtype='object')

In [10]:
# Check the shape of the dataframe
stock_df.shape

(1000, 3)

In [11]:
# Check if any missing values are present in the dataframe
stock_df.isnull().sum()

Interest Rates    0
Employment        0
S&P 500 Price     0
dtype: int64

In [12]:
stock_df = stock_df.dropna()

In [13]:
stock_df.dtypes

Interest Rates    float64
Employment        float64
S&P 500 Price     float64
dtype: object

In [14]:
# check if there are any Null values
sns.heatmap(stock_df.isnull(), yticklabels = False, cbar = False, cmap="Blues")
plt.plot()

[]

In [15]:
sns.jointplot(x = 'Employment', y = 'S&P 500 Price', data=stock_df)
plt.plot()

[]

In [16]:
sns.jointplot(x = 'Interest Rates',y = 'S&P 500 Price', data = stock_df);
plt.plot()

[]

In [17]:
sns.pairplot(stock_df);
plt.plot()

[]

In [18]:
y = stock_df['S&P 500 Price']

In [19]:
X = stock_df[['Interest Rates', 'Employment']]

In [20]:
# X = np.array(X)

In [21]:
# y = np.array(y)

# TASK #3: TRAIN A LINEAR LEARNER MODEL USING SAGEMAKER TO SOLVE MULTIPLE LINEAR REGRESSION PROBLEM

In [22]:
# Convert the datatype to float32 (to run Linear Learner successfully)
X = np.array(X).astype('float32')
y = np.array(y).astype('float32')

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.2)

In [23]:
# Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python
# Boto3 allows Python developer to write software that makes use of services like Amazon S3 and Amazon EC2

import sagemaker
import boto3

# Let's create a Sagemaker session
sagemaker_session = sagemaker.Session()

# Let's define the S3 bucket and prefix that we want to use in this session
# bucket = 'aws-ml-10days' # bucket named 'aws-ml-10days' was created beforehand

bucket = sagemaker_session.default_bucket() 
prefix = 'multi_linear_learner_sp500' # prefix is the subfolder within the bucket.

# Let's get the execution role for the notebook instance. 
# This is the IAM role that you created when you created your notebook instance. You pass the role to the training job.
# Note that AWS Identity and Access Management (IAM) role that Amazon SageMaker can assume to perform tasks on your behalf (for example, reading training results, called model artifacts, from the S3 bucket and writing training results to Amazon S3). 
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::279288473542:role/service-role/AmazonSageMaker-ExecutionRole-20230912T204102


In [24]:
X_train.shape

(800, 2)

In [25]:
y_train.shape

(800,)

In [26]:
# y_train = y_train[:,0]

In [27]:
y_train.shape

(800,)

In [28]:
import io # The io module allows for dealing with various types of I/O (text I/O, binary I/O and raw I/O). 
import numpy as np
import sagemaker.amazon.common as smac # sagemaker common libary

# Code below converts the data in numpy array format to RecordIO format
# This is the format required by Sagemaker Linear Learner 

buf = io.BytesIO() # create an in-memory byte array (buf is a buffer I will be writing to)
smac.write_numpy_to_dense_tensor(buf, X_train, y_train)
buf.seek(0) 
# When you write to in-memory byte arrays, it increments 1 every time you write to it
# Let's reset that back to zero 


0

In [29]:
import os

# Code to upload RecordIO data to S3
 
# Key refers to the name of the file    
key = 'linear-train-data'

# The following code uploads the data in record-io format to S3 bucket to be accessed later for training
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)

# Let's print out the training data location in s3
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

uploaded training data location: s3://sagemaker-us-east-1-279288473542/multi_linear_learner_sp500/train/linear-train-data


/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [30]:
# create an output placeholder in S3 bucket to store the linear learner output

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

Training artifacts will be uploaded to: s3://sagemaker-us-east-1-279288473542/multi_linear_learner_sp500/output


In [31]:
# Note that this code leverages the new SageMaker SDK 2.0
# Check this for the list of changes from AWS SageMaker SDK 1.0 to 2.0: https://sagemaker.readthedocs.io/en/stable/v2.html

# This code is used to get the training container of sagemaker built-in algorithms
# all we have to do is to specify the name of the algorithm that we want to use

# Let's obtain a reference to the linearLearner container image
# Note that all regression models are named estimators
# You don't have to specify (hardcode) the region, get_image_uri will get the current region name using boto3.Session
container = sagemaker.image_uris.retrieve("linear-learner", boto3.Session().region_name)


# This is using the old AWS SageMAker SDK 1.0 (You need to use get_image_uri and note that attribute order is different as well)
# from sagemaker.amazon.amazon_estimator import get_image_uri
# container = get_image_uri(boto3.Session().region_name, 'linear-learner')

In [33]:
# We have pass in the container, the type of instance that we would like to use for training 
# output path and sagemaker session into the Estimator. 
# We can also specify how many instances we would like to use for training

linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       instance_count = 1, 
                                       instance_type = 'ml.m4.xlarge',
                                       output_path = output_location,
                                       sagemaker_session = sagemaker_session,
                                       use_spot_instances = True,
                                       max_run = 300,
                                       max_wait = 600)


# We can tune parameters like the number of features that we are passing in, type of predictor like 'regressor' or 'classifier', mini batch size, epochs
# Train 32 different versions of the model and will get the best out of them (built-in parameters optimization!)

linear.set_hyperparameters(feature_dim = 2,
                           predictor_type = 'regressor',
                           mini_batch_size = 100,
                           epochs = 100,
                           num_models = 32,
                           loss = 'absolute_loss')

# Now we are ready to pass in the training data from S3 to train the linear learner model

linear.fit({'train': s3_train_data})

# Let's see the progress using cloudwatch logs

INFO:sagemaker:Creating training-job with name: linear-learner-2023-09-20-00-58-01-042


2023-09-20 00:58:01 Starting - Starting the training job...
2023-09-20 00:58:25 Starting - Preparing the instances for training.........
2023-09-20 00:59:44 Downloading - Downloading input data......
2023-09-20 01:00:35 Training - Downloading the training image...
2023-09-20 01:01:31 Training - Training image download completed. Training in progress....
2023-09-20 01:02:01 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[09/20/2023 01:01:47 INFO 140100470376256] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 

# TASK #4: DEPLOY AND TEST THE TRAINED LINEAR LEARNER MODEL 

In [34]:
# Deploying the model to perform inference 

# Content type overrides the data that will be passed to the deployed model, since the deployed model expects data in text/csv format.
# Serializer accepts a single argument, the input data, and returns a sequence of bytes in the specified content type
# Deserializer accepts two arguments, the result data and the response content type, and return a sequence of bytes in the specified content type.

from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer


linear_regressor = linear.deploy(initial_instance_count = 1,
                                 instance_type = 'ml.m4.xlarge',
                                 serializer=CSVSerializer(),
                                 deserializer=JSONDeserializer())

INFO:sagemaker:Creating model with name: linear-learner-2023-09-20-01-02-44-833
INFO:sagemaker:Creating endpoint-config with name linear-learner-2023-09-20-01-02-44-833
INFO:sagemaker:Creating endpoint with name linear-learner-2023-09-20-01-02-44-833


------------!

In [35]:
# making prediction on the test data

result = linear_regressor.predict(X_test)

In [36]:
result # results are in Json format

{'predictions': [{'score': 2126.13134765625},
  {'score': 2550.06787109375},
  {'score': 2435.325439453125},
  {'score': 2108.84765625},
  {'score': 2428.021484375},
  {'score': 2697.2978515625},
  {'score': 2211.5830078125},
  {'score': 2286.111083984375},
  {'score': 2528.982177734375},
  {'score': 2390.935302734375},
  {'score': 2068.31884765625},
  {'score': 2314.10009765625},
  {'score': 2425.904296875},
  {'score': 1966.48388671875},
  {'score': 2408.206298828125},
  {'score': 2518.120361328125},
  {'score': 2537.08349609375},
  {'score': 2405.125},
  {'score': 2148.0361328125},
  {'score': 2533.74365234375},
  {'score': 2124.16357421875},
  {'score': 2332.2001953125},
  {'score': 2536.669677734375},
  {'score': 2432.58642578125},
  {'score': 2178.431640625},
  {'score': 2480.91259765625},
  {'score': 2283.654052734375},
  {'score': 2408.903076171875},
  {'score': 2330.151123046875},
  {'score': 2195.69189453125},
  {'score': 2032.510498046875},
  {'score': 2257.86279296875},
  {

In [37]:
# Since the result is in json format, we access the scores by iterating through the scores in the predictions

predictions = np.array([r['score'] for r in result['predictions']])

In [38]:
predictions

array([2126.13134766, 2550.06787109, 2435.32543945, 2108.84765625,
       2428.02148438, 2697.29785156, 2211.58300781, 2286.11108398,
       2528.98217773, 2390.93530273, 2068.31884766, 2314.10009766,
       2425.90429688, 1966.48388672, 2408.20629883, 2518.12036133,
       2537.08349609, 2405.125     , 2148.03613281, 2533.74365234,
       2124.16357422, 2332.20019531, 2536.66967773, 2432.58642578,
       2178.43164062, 2480.91259766, 2283.65405273, 2408.90307617,
       2330.15112305, 2195.69189453, 2032.51049805, 2257.86279297,
       2277.15966797, 2107.71728516, 2258.13427734, 2075.61132812,
       1877.46472168, 2471.97436523, 2230.78442383, 1824.47705078,
       2347.64990234, 2371.84423828, 2314.30297852, 2440.08007812,
       2181.72412109, 2129.84033203, 2271.98144531, 2338.00537109,
       2362.66552734, 2462.79760742, 1941.21105957, 2467.38867188,
       2476.95898438, 2198.65087891, 2181.40209961, 2440.06542969,
       2131.74462891, 2387.484375  , 2412.78955078, 2402.69091

In [39]:
predictions.shape

(200,)

In [40]:
y_test.shape

(200,)

In [41]:
# Visualize how accurate predictions are relative to y_test
plt.figure(figsize = (12, 6))
plt.scatter(y_test, predictions)

In [42]:
# Delete the end-point
linear_regressor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: linear-learner-2023-09-20-01-02-44-833
INFO:sagemaker:Deleting endpoint with name: linear-learner-2023-09-20-01-02-44-833


# EXCELLENT JOB!